In [1]:
import pandas as pd
import os
import re
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import tqdm
from pyvirtualdisplay import Display
import json

In [2]:
display = Display(visible=0, size=(1920,1080))
display.start()

In [3]:
path = '/home/harong/chromedriver'
driver = webdriver.Chrome(path)

In [4]:
# 여기서 무조건 로그인 해야함. 
driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://naver.com/')

In [5]:
driver.get('https://cafe.naver.com/mbticafe')

In [6]:
driver.find_element_by_xpath('//*[@id="menuLink0"]').click()

In [9]:
def crawl_mbti_data() :
    tmp_li = dict()
    for i in tqdm.tqdm(range(602268,0,-1)) : 
        pg = str(i)
        addr = "https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&boardtype=L&articleid="+str(i)+"&referrerAllArticles=true"
        driver.get(addr)
        try : 
            result = driver.switch_to_alert()
            result.accept()
            result.dismiss()
        except : 
            pass
        driver.switch_to.frame('cafe_main')
        html = driver.page_source
        soup = bs(html, 'html.parser')
        writer = soup.findAll("a",{"class":"m-tcol-c b"}).text.split('(')[0]
        category = soup.findAll("a", {"class": "m-tcol-c"})[3].text
        title = soup.find("span",{"class":"b m-tcol-c"}).text
        article = soup.findAll("div", {"class": "tbody m-tcol-c"})[0].text.replace('\u200b', ' ').replace('\n','').strip()

        tmp_li[i] = {'writer':writer,'category':category,'title':title,'article':article}
        
        if i % 100 == 0 :
            with open('/home/harong/main/dev/crawl_data/crawl_res_{i}.json','w') as f:
                json.dump(tmp_li,f)
            f.close()
            tmp_li = dict()

In [10]:
crawl_mbti_data()

  0%|                                                                                                                                | 0/602268 [00:00<?, ?it/s]/home/harong/anaconda3/envs/mbti/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.alert instead
  
  0%|                                                                                                                                | 0/602268 [00:01<?, ?it/s]


AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [25]:
def crawl_mbti_data():
    pg = str(0)
    
    for i in tqdm.tqdm(range(1, 1000)):
        pg = str(i)
        tmp_df = pd.DataFrame()
        tmp_category = []
        tmp_writer = []
        tmp_title = []
        tmp_link = []
        
        addr = "https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=501&search.cafeId=11856775&search.page=" + pg            

        driver.get(addr)
        driver.switch_to.frame('cafe_main')
        
        html = driver.page_source

        soup = bs(html, 'html.parser')
        
        a_category_list = soup.findAll("div", {"class": "inner_name"})
        a_title_list = soup.findAll("a",{"class":"article"})
        a_writer_list = soup.findAll("a",{"class":"m-tcol-c"}) 
        
        if pg == '1':
            a_title_list = a_title_list[16:]
            a_writer_list = a_writer_list[16:]
            
        for j in range(0, len(a_title_list)):
            tmp_writer.append(a_writer_list[j].text.strip())
            tmp_title.append(a_title_list[j].text.strip())
            tmp_category.append(a_category_list[j].text.strip())
            tmp_link.append("https://cafe.naver.com/mbticafe" + a_title_list[j].get('href').strip())
        
        tmp_df['category'] = tmp_category
        tmp_df['title'] = tmp_title
        tmp_df['writer'] = tmp_writer
        tmp_df['link'] = tmp_link
        
        tmp_df.to_csv(f'/home/harong/main/dev/crawl_data/crawl_res_{i}.csv', index=False)
            
            

In [26]:
crawl_mbti_data()

100%|█| 999/999 [20:54<00:00,  1.2


In [30]:
addr = "https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&boardtype=L&articleid=602225&referrerAllArticles=true"
driver.get(addr)
# driver.switch_to.frame('cafe_main')
html = driver.page_source
soup = bs(html, "html.parser")
# category = soup.findAll("a", {"class": "link_board"})
# title = soup.findAll("h3",{"class":"title_text"})
# writer = soup.find("div",{"Writer_Info"})
# article = soup.findAll("div", {"class": "tbody m-tcol-c"})
print(category,title,writer,article)

[] [] None []


In [29]:
def crawl_article(path):
    file_list = os.listdir(path)
    for file in tqdm.tqdm(file_list):
        file_num = file.split('.')[0].split('_')[-1]
        df = pd.read_csv(path + '/' + file)
        tmp_df = pd.DataFrame()
        
        row_len = len(df['category'])
        tmp_article = []
        tmp_category = []
        tmp_writer = []
        tmp_title = []
        tmp_link = []
        for r_idx in range(0, row_len):
            try:
                row = df.iloc[r_idx]
                if row['category'] == '가입 인사 (등업 필수)':
                    continue

                driver.get(row['link'])
                driver.switch_to.frame('cafe_main')
                html = driver.page_source
                soup = bs(html, 'html.parser')
                article = soup.findAll("div", {"class": "tbody m-tcol-c"})[0].text.replace('\u200b', ' ').replace('\n','').strip()
                if article.strip() == '':
                    continue

                tmp_article.append(article)
                tmp_category.append(row['category'])
                tmp_writer.append(row['writer'])
                tmp_link.append(row['link'])
                tmp_title.append(row['title'])
            except:
                continue
        
        tmp_df['category'] = tmp_category
        tmp_df['title'] = tmp_title
        tmp_df['writer'] = tmp_writer
        tmp_df['link'] = tmp_link
        tmp_df['article'] = tmp_article
        
        tmp_df.to_csv(f'/home/harong/main/dev/crawl_data/crawl_res_{file_num}_fin.csv', index=False)
                
            

In [31]:
path = '/home/harong/main/dev/crawl_data/article'
crawl_article(path)

0it [00:00, ?it/s]
